In [2]:
import yaml
import os
import psycopg2
import redis
from dotenv import load_dotenv
import json
import hashlib
from datetime import datetime

In [3]:
load_dotenv()

True

In [4]:
pg_user = os.getenv('POSTGRES_USER')
pg_password = os.getenv('POSTGRES_PASSWORD')
pg_db = os.getenv('POSTGRES_DB')
pg_host = os.getenv('POSTGRES_HOST', 'localhost')
pg_port = os.getenv('POSTGRES_PORT', '5432')

redis_host = os.getenv('REDIS_HOST', 'localhost')
redis_port = os.getenv('REDIS_PORT', '6379')
redis_password = os.getenv('REDIS_PASSWORD')

In [7]:
# Load database schema from YAML file
with open('../database_schema.yaml', 'r') as file:
    schema = yaml.safe_load(file)

In [8]:
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_db,
    user=pg_user,
    password=pg_password,
    port=pg_port
)

In [9]:
pg_cursor = pg_conn.cursor()
print("Successfully connected to PostgreSQL.")

Successfully connected to PostgreSQL.


In [11]:
redis_conn = redis.Redis(
    host=redis_host,
    port=int(redis_port),
    password=redis_password,
    decode_responses=True
)

redis_conn.ping()  # Check if connection is active
print("Successfully connected to Redis.")

Successfully connected to Redis.


In [32]:
def generate_create_table_sql(table_name, table_def):
    columns = table_def.get('columns', {})
    
    # Generate column definitions
    column_defs = []
    primary_keys = []
    
    for col_name, col_def in columns.items():
        # Basic column definition
        col_sql = f"\"{col_name}\" {col_def['type']}"
        
        # Add constraints
        if col_def.get('primary_key', False) is True:
            primary_keys.append(col_name)
            # Add PRIMARY KEY constraint to the column if it's a single-column primary key
            if len(primary_keys) == 1:  # This is the first PK column we've seen
                col_sql += " PRIMARY KEY"
        
        if col_def.get('nullable', True) is False:
            col_sql += " NOT NULL"
            
        if 'default' in col_def:
            col_sql += f" DEFAULT {col_def['default']}"
            
        if 'check' in col_def:
            col_sql += f" CHECK ({col_name} {col_def['check']})"
            
        if 'references' in col_def:
            ref_table, ref_col = col_def['references'].split('.')
            col_sql += f" REFERENCES \"{ref_table}\"(\"{ref_col}\")"
            
        column_defs.append(col_sql)
    
    # For composite primary keys (more than one column), add a separate PRIMARY KEY constraint
    if len(primary_keys) > 1:
        pk_cols = ', '.join(f'"{pk}"' for pk in primary_keys)
        column_defs.append(f"PRIMARY KEY ({pk_cols})")
    elif 'primary_key' in table_def:
        # Handle explicit composite primary key definition from table def
        if isinstance(table_def['primary_key'], list):
            pk_cols = ', '.join(f'"{pk}"' for pk in table_def['primary_key'])
            column_defs.append(f"PRIMARY KEY ({pk_cols})")
    
    # Construct full SQL
    create_sql = f"CREATE TABLE IF NOT EXISTS \"{table_name}\" (\n"
    create_sql += ",\n".join(column_defs)
    create_sql += "\n);"
    
    return create_sql

In [33]:
def initialize_postgres():
    # Get PostgreSQL table definitions from schema
    postgres_tables = schema.get('postgres', {}).get('tables', {})
    
    # Define the exact order for table creation based on dependencies
    table_creation_order = [
        "CHAIN_DETAILS",
        "USER_AUTH",
        "AGENT",
        "UNPUBLISHED_AGENT",
        "PUBLISHED_AGENT",
        "METADATA",
        "CONTRACT_DETAILS",
        "BLOCKCHAIN_AGENT_DATA",
        "ACCESS_LEVEL_TABLE",
        "NFT_ACCESS",
        "USERS",
        "CONVERSATIONS",
        "MESSAGES"
    ]
    
    # Create extension for UUID support
    pg_cursor.execute("CREATE EXTENSION IF NOT EXISTS pgcrypto;")
    
    # Create tables in the specified order
    for table_name in table_creation_order:
        if table_name not in postgres_tables:
            print(f"Warning: Table {table_name} specified in creation order but not found in schema")
            continue
            
        table_def = postgres_tables[table_name]
        create_table_sql = generate_create_table_sql(table_name, table_def)
        print(f"Creating table {table_name}...")
        
        try:
            pg_cursor.execute(create_table_sql)
        except Exception as e:
            print(f"Error creating table {table_name}: {e}")
            print(f"SQL that failed: {create_table_sql}")
            pg_conn.rollback()
            raise
    
    # Commit all changes
    pg_conn.commit()
    print(f"Successfully created all PostgreSQL tables.")

In [34]:
def initialize_redis():
    # Get Redis key definitions from schema
    redis_keys = schema.get('redis', {}).get('keys', {})
    
    # Create sample template for each Redis key type
    for key_name, key_def in redis_keys.items():
        key_pattern = key_def.get('key_pattern', '')
        key_type = key_def.get('type', 'hash')
        ttl = key_def.get('ttl', 0)
        fields = key_def.get('fields', [])
        
        # Create documentation keys with examples to help developers
        doc_key = f"schema:{key_name}:doc"
        
        # Create documentation with field examples
        doc_data = {
            'key_pattern': key_pattern,
            'type': key_type,
            'ttl': ttl,
            'fields': [{'name': field.get('name'), 'description': field.get('description')} for field in fields],
            'example': key_pattern.format(user_id='user123', agent_id='agent456')
        }
        
        redis_conn.set(doc_key, json.dumps(doc_data))
        print(f"Created Redis schema documentation for {key_name}")
    
    # Create metadata about synchronization system
    sync_config = schema.get('synchronization', {})
    redis_conn.set('schema:sync:config', json.dumps(sync_config))
    print("Created Redis synchronization configuration")

In [35]:
initialize_postgres()


Creating table CHAIN_DETAILS...
Creating table USER_AUTH...
Creating table AGENT...
Creating table UNPUBLISHED_AGENT...
Creating table PUBLISHED_AGENT...
Creating table METADATA...
Creating table CONTRACT_DETAILS...
Creating table BLOCKCHAIN_AGENT_DATA...
Creating table ACCESS_LEVEL_TABLE...
Creating table NFT_ACCESS...
Creating table USERS...
Creating table CONVERSATIONS...
Creating table MESSAGES...
Successfully created all PostgreSQL tables.


In [36]:
initialize_redis()

Created Redis schema documentation for USER_SESSION
Created Redis schema documentation for WORKFLOW_LIVE_EDITOR
Created Redis synchronization configuration
